AI Agent: Textbook to Notebook Notes Generator 

In [1]:
import os
import json
import gradio as gr
import textwrap
import asyncio
try:
    import fitz  # PyMuPDF
except Exception:
    fitz = None
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pydantic import BaseModel
from fpdf import FPDF
from datetime import datetime

In [2]:

load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

print("\n=== 🔍 API Configuration Check ===")
if openai_api_key:
    print(f" OpenAI API Key Loaded: {openai_api_key[:8]}****")
else:
    print(" Missing OpenAI API Key. Please set it in .env")

if google_api_key:
    print(f" Google API Key Loaded: {google_api_key[:8]}****")
else:
    print(" Missing Google API Key. Please set it in .env")


=== 🔍 API Configuration Check ===
 OpenAI API Key Loaded: sk-proj-****
 Google API Key Loaded: AIzaSyDn****


In [4]:
ollama_client = AsyncOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

gemini_client = AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [5]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    clarity_score: int
    accuracy_score: int

In [6]:
def chunk_text(text: str, max_chars: int = 3000, overlap: int = 300):
    """Split extracted text into overlapping chunks suitable for LLMs.

    - max_chars: rough upper bound per chunk to keep prompts manageable
    - overlap: carry some context to the next chunk to preserve continuity
    """
    text = text or ""
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks = []
    current = ""

    for para in paragraphs:
        if len(current) + len(para) + 2 <= max_chars:
            current = (current + "\n\n" + para) if current else para
        else:
            if current:
                chunks.append(current)
            # if a single paragraph is too large, hard-split it
            if len(para) > max_chars:
                start = 0
                while start < len(para):
                    end = min(start + max_chars, len(para))
                    chunks.append(para[start:end])
                    start = max(0, end - overlap)
                current = ""
            else:
                current = para

    if current:
        chunks.append(current)

    # add overlap between chunks
    if overlap > 0 and len(chunks) > 1:
        overlapped = []
        for i, ch in enumerate(chunks):
            if i == 0:
                overlapped.append(ch)
            else:
                prev = overlapped[-1]
                tail = prev[-overlap:]
                overlapped.append(tail + ch if tail else ch)
        return overlapped

    return chunks


In [7]:
async def generate_notes(text_chunk: str, retries=2, feedback=""):
    """Generate structured notes for a given text chunk using Ollama model."""
    system_prompt = (
        "You are an AI Developer Agent specialized in academic note generation. "
        "Analyze the input text and produce structured, markdown-formatted notes. "
        "Highlight key ideas, maintain precision, and ensure clarity. "
        "Use professional, educational tone suitable for developers or researchers."
    )

    if feedback:
        user_prompt = f"Refine the previous output using this feedback:\n{feedback}\n\nOriginal Text:\n{text_chunk}"
    else:
        user_prompt = f"Generate professional academic notes for:\n{text_chunk}"

    try:
        response = await ollama_client.chat.completions.create(
            model="llama3:8b",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        notes = response.choices[0].message.content
    except Exception as e:
        print(f"⚠️ Ollama generation error: {e}")
        return f"Error generating notes: {e}"

    if retries > 0:
        evaluation = await evaluate_notes(text_chunk, notes)
        if not evaluation.is_acceptable or (evaluation.clarity_score < 3 or evaluation.accuracy_score < 3):
            print(f"🔁 Re-evaluating due to low quality: {evaluation.feedback}")
            return await generate_notes(text_chunk, retries - 1, evaluation.feedback)
        else:
            print("✅ Evaluation passed.")
    return notes

In [8]:
async def evaluate_notes(text_chunk: str, notes: str) -> Evaluation:
    """Evaluate generated notes with Gemini for accuracy and clarity."""
    prompt = (
        "Evaluate these notes based on accuracy, clarity, and completeness. "
        "Return a JSON with: is_acceptable (bool), feedback (string), clarity_score (1–5), accuracy_score (1–5). "
        f"\n\n--- Original Text ---\n{text_chunk}\n\n--- Notes ---\n{notes}"
    )

    try:
        response = await gemini_client.chat.completions.create(
            model="gemini-2.5-flash",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
        )
        data = json.loads(response.choices[0].message.content)
        return Evaluation(**data)
    except Exception as e:
        print(f"⚠️ Gemini evaluation error: {e}")
        # FIXED
    return Evaluation(
        is_acceptable=True,
        feedback=f"Fallback: {e}",
        clarity_score=5,
        accuracy_score=5
)


In [9]:
async def generate_final_summary(all_notes: str):
    """Generate a concise, professional executive summary."""
    print("🧠 Generating executive summary...")
    prompt = (
        "Summarize all provided notes into a clean, professional overview in Markdown. "
        "Focus on major themes, concepts, and takeaways suitable for AI developers or educators.\n\n"
        f"{all_notes}"
    )
    try:
        response = await gemini_client.chat.completions.create(
            model="gemini-2.5-flash",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"⚠️ Gemini summary error: {e}")
        return "Error generating summary."

In [10]:
class StyledPDF(FPDF):
    """Stylish PDF generator for developer-grade reports."""
    def header(self):
        self.set_font('Arial', 'I', 8)
        self.set_text_color(100, 100, 100)
        self.cell(0, 10, 'AI Agent Developer Notes Generator', 0, 0, 'C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128, 128, 128)
        page_num_text = f"Page {self.page_no()} | AI Agent Developer"
        self.cell(0, 10, page_num_text.encode('latin-1', 'replace').decode('latin-1'), 0, 0, 'C')

    def set_title_page(self, title: str, subtitle: str = ""):
        self.add_page()
        self.set_font('Arial', 'B', 22)
        self.set_text_color(0, 51, 102)
        # Apply encoding fix
        safe_title = title.encode('latin-1', 'replace').decode('latin-1')
        self.cell(0, 15, safe_title, 0, 1, 'C')
        if subtitle:
            self.ln(4)
            self.set_font('Arial', '', 12)
            self.set_text_color(80, 80, 80)
            # Apply encoding fix
            safe_subtitle = subtitle.encode('latin-1', 'replace').decode('latin-1')
            self.cell(0, 10, safe_subtitle, 0, 1, 'C')
        self.ln(10)

    def add_table_of_contents(self, sections):
        if not sections:
            return
        self.add_page()
        self.set_font('Arial', 'B', 16)
        self.set_text_color(51, 102, 153)
        self.cell(0, 10, 'Table of Contents', 0, 1)
        self.ln(2)
        self.set_font('Arial', '', 11)
        self.set_text_color(0, 0, 0)
        for idx, sec in enumerate(sections, start=1):
            title = sec.get('title', f'Section {idx}')
            # Apply encoding fix
            toc_entry = f"{idx}. {title}".encode('latin-1', 'replace').decode('latin-1')
            self.multi_cell(0, 6, toc_entry)
        self.ln(4)

    def process_markdown_content(self, content):
        self.set_font('Arial', '', 11)
        for line in content.split('\n'):
            # Apply encoding fix to the line before processing
            safe_line = line.encode('latin-1', 'replace').decode('latin-1')
            
            if line.startswith('# '):
                self.ln(8)
                self.set_font('Arial', 'B', 16)
                self.set_text_color(0, 51, 102)
                self.cell(0, 10, safe_line[2:], 0, 1)
            elif line.startswith('## '):
                self.ln(6)
                self.set_font('Arial', 'B', 13)
                self.set_text_color(51, 102, 153)
                self.cell(0, 8, safe_line[3:], 0, 1)
            else:
                self.set_font('Arial', '', 11)
                self.set_text_color(0, 0, 0)
                self.multi_cell(0, 6, safe_line)
                self.ln(1)

In [11]:
def create_styled_pdf_file(notes_markdown: str, source_filename: str) -> str:
    """
    Create a beautifully formatted PDF with enhanced styling.
    """
    title = os.path.splitext(os.path.basename(source_filename))[0].replace('_', ' ').title()
    output_filename = f"{os.path.splitext(source_filename)[0]}_styled_notes.pdf"
    output_filename_abs = os.path.abspath(output_filename)
    
    # Create styled PDF instance
    pdf = StyledPDF()
    
    # Set up document properties
    pdf.set_title("AI Generated Textbook Notes")
    
    # Create title page
    pdf.set_title_page(
        title=title,
        subtitle="Generated by AI Textbook Notes Generator"
    )
    
    # Parse content to extract sections for table of contents
    sections = []
    lines = notes_markdown.split('\n')
    current_section = None
    
    for line in lines:
        line = line.strip()
        if line.startswith('# '):
            if current_section:
                sections.append(current_section)
            current_section = {'title': line[2:].strip()}
        elif line.startswith('## ') and current_section:
            current_section['title'] = line[3:].strip()
    
    if current_section:
        sections.append(current_section)
    
    # Add table of contents
    pdf.add_table_of_contents(sections)
    
    # Process and add content
    try:
        pdf.process_markdown_content(notes_markdown)
        
        # Save the PDF
        pdf.output(output_filename_abs)
        print(f"✨ Beautiful PDF created successfully: {output_filename_abs}")
        return output_filename_abs
        
    except Exception as e:
        print(f"Error creating styled PDF: {e}")
        
        # Fallback: Create error report with same styling
        try:
            error_pdf = StyledPDF()
            error_pdf.add_page()
            
            # Error title page
            error_pdf.set_title_page(
                title="Error Report",
                subtitle="PDF Generation Failed"
            )
            
            # Error details
            error_section = "Error Details\n\n" + str(e)
            error_pdf.process_markdown_content(error_section)
            
            error_pdf.output(output_filename_abs)
            print(f"Error report saved: {output_filename_abs}")
            return output_filename_abs
            
        except Exception as e2:
            print(f"CRITICAL: Failed to create error report: {e2}")
            return None


In [12]:
async def process_textbook(file, progress=gr.Progress()):
    """
    Enhanced main orchestrator function with better progress tracking.
    """
    if file is None:
        return None

    if not 'PYMUPDF_OK' in globals() or not PYMUPDF_OK:
        print("❌ PyMuPDF not available. Please install 'PyMuPDF' and restart the kernel.")
        return None

    pdf_file_path = file.name
    
    # STEP 1: Extract Text with PyMuPDF (fitz)
    print("Step 1/5: Extracting Text...")
    progress(0, desc="Step 1/5: Extracting Text...")
    full_text = ""
    try:
        doc = fitz.open(pdf_file_path)
        num_pages = len(doc)
        for i, page in enumerate(doc):
            progress((i + 1) / num_pages, desc=f"Extracting from Page {i + 1}/{num_pages}")
            full_text += page.get_text() + "\n"
        doc.close()
    except Exception as e:
        print(f"Error extracting text with PyMuPDF: {e}")
        return None

    if not full_text.strip():
        print("No text could be extracted from the PDF.")
        return None

    print(f"✅ Text extraction complete. Total characters: {len(full_text)}")
    
    # STEP 2: Chunk Text
    chunks = chunk_text(full_text)
    num_chunks = len(chunks)
    all_notes = []
    
    # STEP 3: Generate Notes
    print(f"Step 2/5: Generating notes from {num_chunks} text chunks...")
    progress(0, desc="Step 2/5: Generating Notes...")
    for i, chunk in enumerate(chunks):
        progress((i + 1) / num_chunks, desc=f"Processing Chunk {i + 1}/{num_chunks}")
        notes_chunk = await generate_notes(chunk)
        all_notes.append(notes_chunk)
        
    combined_notes = "\n\n---\n\n".join(all_notes)
    print("✅ Notes generation complete")

    # STEP 4: Generate Final Summary
    print("Step 3/5: Generating Final Summary...")
    progress(0.8, desc="Step 3/5: Generating Final Summary...")
    final_summary = await generate_final_summary(combined_notes)
    print("✅ Summary generation complete")
    
    final_markdown = f"# Executive Summary\n\n{final_summary}\n\n---\n\n# Detailed Notes\n\n{combined_notes}"

    # STEP 5: Create Styled PDF
    print("Step 4/5: Creating Beautiful PDF...")
    progress(0.9, desc="Step 4/5: Creating Beautiful PDF...")
    pdf_path = create_styled_pdf_file(final_markdown, pdf_file_path)
    
    progress(1.0, desc="Step 5/5: Complete!")
    print("🎉 All processing complete!")
    
    return pdf_path

In [13]:
async def create_notes_interface(file, progress=gr.Progress(track_tqdm=True)):
    """Enhanced interface function with better feedback"""
    if file is not None:
        try:
            result = await process_textbook(file, progress)
            if result:
                return result
            else:
                # Return None to File output when processing fails
                return None
        except Exception as e:
            # Return None to avoid File component stat on error strings
            print(f"❌ An error occurred: {str(e)}")
            return None
    # No file uploaded; return None for File output
    return None

In [14]:
# --- Safe PyMuPDF Import Check ---
PYMUPDF_OK = False
try:
    if fitz is None or not hasattr(fitz, "open"):
        raise ImportError(
            "\n❌ Wrong 'fitz' module detected or not installed.\n"
            "Please run inside your virtual environment:\n"
            "   pip uninstall fitz -y\n"
            "   pip install PyMuPDF\n"
        )
    PYMUPDF_OK = True
    print("✅ Correct PyMuPDF detected! Using:", getattr(fitz, "__doc__", "PyMuPDF").splitlines()[0])
except ImportError as e:
    # Warn but do not exit; allow UI to load so user can fix env and retry
    print(e)
# --------------------------------


✅ Correct PyMuPDF detected! Using: PyMuPDF 1.26.5: Python bindings for the MuPDF 1.26.10 library (rebased implementation).


In [15]:
with gr.Blocks(css="""
.gradio-container {
    max-width: 850px !important;
    margin: auto;
    font-family: 'Segoe UI', sans-serif;
}
.title {
    text-align: center;
    color: #003366;
    font-size: 28px;
    margin-bottom: 8px;
}
.description {
    text-align: center;
    color: #555;
    margin-bottom: 25px;
}
.footer {
    text-align: center;
    color: #888;
    font-size: 12px;
    margin-top: 40px;
}
""") as iface:
    
    gr.HTML("""
    <div class="title">🤖 AI Agent Developer: Textbook → Notes Generator</div>
    <div class="description">
        Built for AI developers and educators to auto-generate well-structured academic notes 
        and summaries from textbook PDFs.
    </div>
    """)

    file_input = gr.File(label="📘 Upload Textbook (PDF)", file_types=[".pdf"])
    output_file = gr.File(label="📥 Download Generated Notes (PDF)")
    submit_btn = gr.Button("⚙️ Generate Notes", variant="primary")

    submit_btn.click(
        fn=create_notes_interface,
        inputs=[file_input],
        outputs=[output_file],
        show_progress="full"
    )

    gr.HTML("""
    <div class="footer">
        © 2025 AI Agent Developer Suite | Powered by Ollama & Gemini APIs
    </div>
    """)

if __name__ == "__main__":
    print("🚀 Launching AI Agent Developer Interface...")
    iface.launch(server_name="127.0.0.1", share=False, show_error=True)

🚀 Launching AI Agent Developer Interface...
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Step 1/5: Extracting Text...
✅ Text extraction complete. Total characters: 48832
Step 2/5: Generating notes from 24 text chunks...
🔁 Re-evaluating due to low quality: The notes are very clear and easy to understand, providing good summaries of the content. However, there is a significant accuracy issue regarding the preface: the notes state "No preface provided in this edition," while the original text's Table of Contents clearly lists "Preface to the Expanded and Updated Edition." Additionally, while all chapters are mentioned, the notes do not consistently list each individual chapter as a distinct heading or bullet point, particularly within Step I, II, and III (e.g., 'Cautions and Comparisons,' 'The Low-Information Diet,' 'Outsourcing Life' are summarized within broader sections rather than given their own distinct entries as they appear in the original TOC). This affects the completeness and precision of the outline.
🔁 Re-evaluating due to low quality: The notes are excellent in t